Ref doc link-https://huggingface.co/docs/transformers/tasks/language_modeling#causal-language-modeling

Installing all the required libraries

In [1]:
pip install transformers datasets evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.9 MB/s eta 0:00:00


Importing required libraries

In [2]:
import numpy as np
import pandas as pd

Loading wikitext data from huggingface datasets and taking only 10000 instances . Note if we take 1000 instances only and run the notebook.. The Perplexity score will be slighly lower

In [3]:
from datasets import load_dataset

wikitext_ds = load_dataset(path = "wikitext", name = "wikitext-2-raw-v1", split = "train[:10000]")

wikitext_ds

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

Dataset({
    features: ['text'],
    num_rows: 10000
})

Splitting the dataset’s train split into a train and test set with the train_test_split method

In [4]:
wikitext_ds = wikitext_ds.train_test_split(test_size = 0.2)

wikitext_ds

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 8000
    })
    test: Dataset({
        features: ['text'],
        num_rows: 2000
    })
})

Taking a look at few instances

In [5]:
wikitext_ds['train'][0]

{'text': " Saprang was considered one of the top contenders to lead the army and the junta after CNS @-@ leader Sonthi Boonyaratkalin 's mandatory retirement in 2007 . However , in September 2007 he was demoted to be Deputy Permanent Secretary of the Defense Ministry , while his rival , General Anupong Paochinda , was promoted to lead the Army . As a result , Saprang retired from the Army in 2010 . \n"}

In [6]:
wikitext_ds['train'][35]

{'text': ' Not Quite Hollywood had its worldwide premiere at the Melbourne International Film Festival on 28 July 2008 , and was screened at the Australian Centre for the Moving Image . Its Australia @-@ wide release was a month later , on 28 August 2008 , and it had its overseas premiere at the Toronto Film Festival on 7 September 2008 , where distribution rights were secured for the United Kingdom , Canada , France , Russia , Germany and Benelux . The film was also screened at the Austin , Sitges , Warsaw , Helsinki and Stockholm International Film Festivals in 2008 , and featured at the London Film Festival on 25 October 2008 . \n'}

We would be starting to preprocess train data. First, we  convert the text in the input into IDs the model can make sense of, using a tokenizer.Loading DistilGPT2 tokenizer to process the text subfield:

In [17]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilgpt2")

tokenizer

GPT2TokenizerFast(name_or_path='distilgpt2', vocab_size=50257, model_max_length=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	50256: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
}

Here is a first preprocessing function to tokenize the text

In [9]:
def preprocess_function(examples):
    return tokenizer(examples['text'])

To apply this preprocessing function over the entire dataset,we use the 🤗 Datasets map method. We can speed up the map function by setting batched=True to process multiple elements of the dataset at once, and increasing the number of processes with num_proc. Remove any columns you don’t need:

In [10]:
tokenized_wikitext_ds = wikitext_ds.map(
    preprocess_function,
    batched = True,
    num_proc = 4,
    remove_columns = wikitext_ds['train'].column_names
)

tokenized_wikitext_ds

Map (num_proc=4):   0%|          | 0/8000 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/2000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 8000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 2000
    })
})

Looking at one instance of tokenization

In [14]:
wikitext_ds["train"][4]

{'text': ' Dirigent proteins are members of a class of proteins that dictate the stereochemistry of a compound synthesized by other enzymes . \n'}

In [15]:
tokenized_wikitext_ds["train"][4]

{'input_ids': [36202,
  47096,
  15568,
  389,
  1866,
  286,
  257,
  1398,
  286,
  15568,
  326,
  27861,
  262,
  10268,
  37074,
  286,
  257,
  13061,
  24983,
  1143,
  416,
  584,
  29120,
  764,
  220,
  198],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1]}

In [16]:
tokenizer.decode(tokenized_wikitext_ds["train"][4]["input_ids"])

' Dirigent proteins are members of a class of proteins that dictate the stereochemistry of a compound synthesized by other enzymes. \n'

This dataset contains the token sequences, but some of these are longer than the maximum input length for the model.

Using  a second preprocessing function to
concatenate all the sequences
split the concatenated sequences into shorter chunks defined by block_size, which should be both shorter than the maximum input length and short enough for your GPU RAM.

In [30]:
examples = tokenized_wikitext_ds['train'][3:6]

print("Keys: ", examples.keys())
print("input_ids", examples['input_ids'])
print("attention_mask", examples['attention_mask'])

Keys:  dict_keys(['input_ids', 'attention_mask'])
input_ids [[383, 779, 286, 311, 65, 355, 262, 3210, 5931, 6194, 329, 1885, 33969, 318, 2233, 284, 449, 9101, 5907, 25845, 672, 4312, 17396, 3754, 837, 508, 973, 428, 28873, 47625, 286, 262, 1438, 336, 571, 1505, 764, 383, 19955, 9133, 1296, 837, 422, 543, 262, 3660, 8950, 290, 2739, 45772, 8312, 1011, 511, 3891, 329, 1885, 33969, 837, 318, 1885, 20473, 1505, 764, 383, 8159, 286, 428, 318, 8627, 2162, 477, 11776, 423, 617, 8722, 2035, 286, 1296, 393, 10794, 764, 383, 2968, 304, 43408, 837, 422, 28053, 120, 222, 26180, 32830, 138, 107, 34703, 26517, 26180, 17394, 139, 229, 139, 234, 35558, 3098, 2488, 12, 31, 937, 620, 418, 393, 4141, 1885, 25147, 500, 837, 991, 468, 46372, 2162, 428, 561, 1612, 366, 27266, 2488, 12, 31, 11868, 366, 837, 290, 318, 4893, 416, 867, 1903, 435, 15245, 1023, 852, 27646, 837, 290, 1885, 33969, 852, 32199, 764, 220, 198], [36202, 47096, 15568, 389, 1866, 286, 257, 1398, 286, 15568, 326, 27861, 262, 10268, 37074,

In [31]:
concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}

print("Keys: ", concatenated_examples.keys())
print("input_ids", concatenated_examples['input_ids'])
print("attention_mask", concatenated_examples['attention_mask'])

Keys:  dict_keys(['input_ids', 'attention_mask'])
input_ids [383, 779, 286, 311, 65, 355, 262, 3210, 5931, 6194, 329, 1885, 33969, 318, 2233, 284, 449, 9101, 5907, 25845, 672, 4312, 17396, 3754, 837, 508, 973, 428, 28873, 47625, 286, 262, 1438, 336, 571, 1505, 764, 383, 19955, 9133, 1296, 837, 422, 543, 262, 3660, 8950, 290, 2739, 45772, 8312, 1011, 511, 3891, 329, 1885, 33969, 837, 318, 1885, 20473, 1505, 764, 383, 8159, 286, 428, 318, 8627, 2162, 477, 11776, 423, 617, 8722, 2035, 286, 1296, 393, 10794, 764, 383, 2968, 304, 43408, 837, 422, 28053, 120, 222, 26180, 32830, 138, 107, 34703, 26517, 26180, 17394, 139, 229, 139, 234, 35558, 3098, 2488, 12, 31, 937, 620, 418, 393, 4141, 1885, 25147, 500, 837, 991, 468, 46372, 2162, 428, 561, 1612, 366, 27266, 2488, 12, 31, 11868, 366, 837, 290, 318, 4893, 416, 867, 1903, 435, 15245, 1023, 852, 27646, 837, 290, 1885, 33969, 852, 32199, 764, 220, 198, 36202, 47096, 15568, 389, 1866, 286, 257, 1398, 286, 15568, 326, 27861, 262, 10268, 37074, 28

In [33]:
block_size = 128

def group_texts(examples):
    # Concatenate all texts in one long input
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}

    total_length = len(concatenated_examples[list(examples.keys())[0]])

    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size

    # Split by chunks of block_size.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

Applying the group_texts function over the entire dataset

In [34]:
wikitext_lm_dataset = tokenized_wikitext_ds.map(group_texts, batched = True, num_proc = 4)

wikitext_lm_dataset

Map (num_proc=4):   0%|          | 0/8000 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/2000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 4006
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1010
    })
})

In [35]:
tokenizer.decode(wikitext_lm_dataset["train"][55]["input_ids"])

' Further south, along the County Wexford coastline, is an area of granite intrusives into more Ordovician and Silurian rocks, like that found in Wales. \n He proposed to his longtime girlfriend, Cuban @-@ American model Yvette Prieto, on Christmas Eve, 2011, and they were married on April 27, 2013, at Bethesda @-@ by @-@ the @-@ Sea Episcopal Church. It was announced on November 30, 2013, that the two were expecting their first child together. Jordan listed his Highland Park mansion for sale in 2012. On February 11, 2014, Prieto gave'

In [36]:
len(wikitext_lm_dataset["train"][55]['input_ids'])

128

In [38]:
tokenizer.decode(wikitext_lm_dataset["train"][545]["input_ids"])

' million U.S. viewers and attained a 3 @.@ 2 / 8 rating / share in the 18 – 49 demographic. Glee maintained its ratings from the previous week, despite all of the other new Wednesday night shows of the season declining by double @-@ digit percentages. It was the eighteenth most watched show in Canada for the week of broadcast, with 1 @.@ 61 million viewers. In the UK, the episode was watched by 2 @.@ 008 million viewers ( 1 @.@ 608 million on E4, and 400 @,@ 000 on E4 + 1 ), becoming the most @-@'

In [39]:
len(wikitext_lm_dataset["train"][545]['input_ids'])

128

Now creating a batch of examples using DataCollatorForLanguageModeling. It’s more efficient to dynamically pad the sentences to the longest length in a batch during collation, instead of padding the whole dataset to the maximum length. Use the end-of-sequence token as the padding token and set mlm=False. This will use the inputs as labels shifted to the right by one element:

MLM (Masked Language Modeling): Typically, DataCollatorForLanguageModeling is used for preparing data for MLM tasks. In MLM, as popularized by models like BERT, random tokens in a sequence are replaced with a special [MASK] token, and the model is trained to predict these masked tokens based on the context provided by the unmasked tokens. Setting mlm=True configures the collator for this type of task.

Causal (Autoregressive) Language Modeling: When you set mlm=False, the data collator is configured for causal or autoregressive language modeling. In this type of task, the model predicts the next token in a sequence given the preceding tokens. It's a different approach compared to MLM, as there's no masking involved; instead, the model learns to generate text. This setting is used for models like GPT (Generative Pretrained Transformer).

Training Implications: The choice between MLM and causal language modeling affects how the input data is prepared and how the model learns. For MLM, the model learns to understand and predict based on bidirectional context (looking at tokens before and after the masked token), while for causal language modeling, the model only looks at tokens that have appeared before the current token in the sequence.

In [40]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer = tokenizer, mlm = False, return_tensors = "tf")

For finetuning a model in TensorFlow, start by setting up an optimizer function, learning rate schedule, and some training hyperparameters

In [41]:
from transformers import create_optimizer, AdamWeightDecay

optimizer = AdamWeightDecay(learning_rate = 2e-5, weight_decay_rate = 0.01)

Loading a DistilGPT2 with TFAutoModelForCausalLM

In [42]:
from transformers import TFAutoModelForCausalLM

model = TFAutoModelForCausalLM.from_pretrained("distilgpt2")

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [43]:
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

1

Converting  datasets to the tf.data.Dataset format with prepare_tf_dataset()

In [44]:
tf_train_set = model.prepare_tf_dataset(
    wikitext_lm_dataset["train"],
    shuffle = True,
    batch_size = 32,
    collate_fn = data_collator,
)

tf_test_set = model.prepare_tf_dataset(
    wikitext_lm_dataset["test"],
    shuffle = False,
    batch_size = 32,
    collate_fn = data_collator,
)

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Configuring  the model for training with compile. Note that Transformers models all have a default task-relevant loss function, so you don’t need to specify one unless you want to:

In [45]:
import tensorflow as tf

model.compile(optimizer = optimizer)

model.summary()

Model: "tfgpt2lm_head_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 transformer (TFGPT2MainLay  multiple                  81912576  
 er)                                                             
                                                                 
Total params: 81912576 (312.47 MB)
Trainable params: 81912576 (312.47 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Logging into account for pushing trained model to Hub

In [46]:
from huggingface_hub import notebook_login

notebook_login()

In [47]:
from transformers.keras_callbacks import PushToHubCallback

callback = PushToHubCallback(
    output_dir = "clm-wikitext-model",
    tokenizer = tokenizer,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
Cloning https://huggingface.co/jinxxx123/clm-wikitext-model into local empty directory.


Finally, For model training,  Calling fit with your training and validation datasets, the number of epochs, and your callback to finetune the model

In [48]:
model.fit(x = tf_train_set, validation_data = tf_test_set, epochs = 3, callbacks = [callback])

Epoch 1/3
  6/125 [>.............................] - ETA: 1:30 - loss: 4.7032

125/125 [==============================] - 134s 943ms/step - loss: 4.2001 - val_loss: 3.9108
Epoch 2/3
125/125 [==============================] - 142s 1s/step - loss: 3.9186 - val_loss: 3.8146
Epoch 3/3
125/125 [==============================] - 141s 1s/step - loss: 3.8062 - val_loss: 3.7666


The quality of language models is often measured in 'perplexity' rather than cross-entropy. For converting to perplexity, we simply raise e to the power of the cross-entropy loss.

In the context of causal language models, a lower perplexity score means the model predictions are closer to the actual progression of words in the text. It implies that the model's understanding of the language structure and word dependencies is better.

In [49]:
import math

eval_loss = model.evaluate(tf_test_set)

print(f"Perplexity: {math.exp(eval_loss):.2f}")

32/32 [==============================] - 10s 310ms/step - loss: 3.7666
Perplexity: 43.23


Inferencing our trained model with different instances

In [50]:
#test_sentence = "The sun"
#test_sentence = "The aircraft"

#test_sentence = "The country"
#test_sentence = "The economic summit was"
test_sentence = "The tournament"

In [51]:
from transformers import pipeline

generator = pipeline("text-generation", model = "clm-wikitext-model")

generator(test_sentence)

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at clm-wikitext-model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'The tournament will be hosted by the Australian Football League. The four regional @-@ league squads – the United States, Mexico and the European – have each finished 5 @.@ 3 @.@ 9 points clear by a score of nine – six'}]

Tokenizing the text and return the input_ids as TensorFlow tensors

In [53]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("clm-wikitext-model")

inputs = tokenizer(test_sentence, return_tensors = "tf").input_ids

inputs

<tf.Tensor: shape=(1, 2), dtype=int32, numpy=array([[ 464, 7756]], dtype=int32)>

Using the generate() method to create the summarization.

In [54]:
from transformers import TFAutoModelForCausalLM

model = TFAutoModelForCausalLM.from_pretrained("clm-wikitext-model")

outputs = model.generate(input_ids = inputs, max_new_tokens = 100, do_sample = True, top_k = 50, top_p = 0.95)

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at clm-wikitext-model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Decoding the generated token ids back into text:

In [55]:
tokenizer.batch_decode(outputs, skip_special_tokens = True)

['The tournament took place on 23 September, with the first ever – first at the University of Chicago. The second – a group stage – between the three teams – was held between 19 August and 22 September, and the first – a final @-@ time tournament between the teams was held between 29 September and 29 September between the three teams was held between 28 September and 29 September. The tournament was held in the name of the UK, the same day that the game was set to take place in the US –']